# Financial Forecasting - Demo

Демонстрация использования класса **FinancialForecaster** для предсказания вероятностей роста акций на 1-20 дней вперед.

## Структура решения:
- `solution.py` - основной класс FinancialForecaster
- `demo.ipynb` - этот ноутбук с примером использования
- `README.md` - документация

## Формат данных:

### Входные данные (train):
**candles.csv**:
```
ticker,begin,open,high,low,close,volume
AFLT,2020-06-19,52.5,53.2,52.1,52.8,1000000
...
```

**news.csv**:
```
publish_date,title,publication
2020-06-19,"Заголовок новости","Полный текст публикации"
...
```

### Выходные данные (submission):
```
ticker,p1,p2,p3,p4,p5,...,p20
AFLT,0.51,0.52,0.48,0.50,0.53,...,0.53
SBER,0.49,0.53,0.51,0.50,0.52,...,0.50
```

**p1-p20** - вероятности роста цены на каждый из 20 дней вперед (одна строка на тикер)

In [1]:
import sys
sys.path.append('.')

from solution import FinancialForecaster
import pandas as pd
import numpy as np

## 1. Инициализация модели

In [2]:
# Создаем экземпляр модели
model = FinancialForecaster(random_state=42)

print("✓ Модель инициализирована")

✓ Модель инициализирована


## 2. Обучение модели

Обучение на исторических данных (candles + news).

In [ ]:
# Пути к обучающим данным
train_candles_path = '../forecast_data/candles.csv'
train_news_path = '../forecast_data/news.csv'

# Обучение модели
model.fit(
    candles_path=train_candles_path,
    news_path=train_news_path
)

print("\n✓ Модель обучена")

ОБУЧЕНИЕ МОДЕЛИ

[1/6] Загрузка данных...
  • Загружено 24197 свечей по 19 тикерам
  • Загружено 25425 новостей

[2/6] Обработка новостей (sentiment analysis)...
  • Средний sentiment: 0.289

[3/6] Feature engineering...
  • Создано 99 фич

[4/6] Кластеризация тикеров...
  • Создано 4 кластеров тикеров

[5/6] Подготовка данных для обучения...
  • Tree features: 75
  • Macro features: 50

[6/6] Обучение ансамбля моделей...
  • Обучение полного ансамбля для p1-p20...
    Модели на горизонт: 2xLGBM + CatBoost + Ridge = 4 модели
    Tree features: 75, Macro features: 50


## 3. Сохранение обученной модели

Сохраняем модель для последующего использования без переобучения.

In [ ]:
# Сохраняем модель
model.save('trained_model.pkl')

print("✓ Модель сохранена в trained_model.pkl")

✓ Модель сохранена: trained_model.pkl
✓ Модель сохранена в trained_model.pkl


## 4. Предсказание на новых данных

Загружаем тестовые данные и генерируем submission.

In [ ]:
# Пути к тестовым данным
test_candles_path = '../forecast_data/candles_2.csv'  # или другой файл
test_news_path = '../forecast_data/news_2.csv'

# Генерируем предсказания
submission = model.predict(
    candles_path=test_candles_path,
    news_path=test_news_path,
    output_path='submission.csv'
)

print("\n✓ Submission создан")

ПРЕДСКАЗАНИЕ

[1/4] Загрузка тестовых данных...
  • Загружено 1745 свечей по 19 тикерам
  • Период: 2025-06-02 - 2025-09-08

[2/4] Обработка новостей...
  • Средний sentiment: 0.329

[3/4] Feature engineering...
  • Создано 99 фич

[4/4] Генерация предсказаний...

✓ Submission сохранен: submission.csv
  • Строк: 19
  • Тикеров: 19

✓ ПРЕДСКАЗАНИЕ ЗАВЕРШЕНО

✓ Submission создан


## 5. Просмотр результатов

In [ ]:
# Смотрим на первые строки submission
print("Первые 10 строк submission:\n")
print(submission.head(10))

print(f"\nВсего строк: {len(submission)}")
print(f"Уникальных тикеров: {submission['ticker'].nunique()}")

# Статистика по вероятностям
print("\nСтатистика вероятностей (p1-p20):")
prob_cols = [f'p{i}' for i in range(1, 21)]
print(submission[prob_cols].describe())

# Примеры предсказаний для конкретных тикеров
print("\n\nПримеры предсказаний:")
for ticker in submission['ticker'].head(3):
    row = submission[submission['ticker'] == ticker].iloc[0]
    print(f"\n{ticker}:")
    print(f"  p1 (1 день):   {row['p1']:.4f}")
    print(f"  p5 (5 дней):   {row['p5']:.4f}")
    print(f"  p10 (10 дней): {row['p10']:.4f}")
    print(f"  p20 (20 дней): {row['p20']:.4f}")

Первые 10 строк submission:

  ticker        p1        p2   p3   p4        p5   p6   p7   p8   p9  ...  \
0   AFLT  0.506860  0.485220  0.5  0.5  0.514339  0.5  0.5  0.5  0.5  ...   
1   ALRS  0.506860  0.485378  0.5  0.5  0.511276  0.5  0.5  0.5  0.5  ...   
2   CHMF  0.506947  0.485466  0.5  0.5  0.512664  0.5  0.5  0.5  0.5  ...   
3   GAZP  0.506860  0.485146  0.5  0.5  0.514542  0.5  0.5  0.5  0.5  ...   
4   GMKN  0.507742  0.481720  0.5  0.5  0.527127  0.5  0.5  0.5  0.5  ...   
5   LKOH  0.506716  0.481117  0.5  0.5  0.511668  0.5  0.5  0.5  0.5  ...   
6   MAGN  0.509900  0.485850  0.5  0.5  0.531792  0.5  0.5  0.5  0.5  ...   
7   MGNT  0.506947  0.488106  0.5  0.5  0.523132  0.5  0.5  0.5  0.5  ...   
8   MOEX  0.506912  0.460588  0.5  0.5  0.483956  0.5  0.5  0.5  0.5  ...   
9   MTSS  0.506716  0.453715  0.5  0.5  0.494776  0.5  0.5  0.5  0.5  ...   

   p11  p12  p13  p14  p15  p16  p17  p18  p19       p20  
0  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.450831  
1  0.

## 6. Альтернативный вариант: загрузка обученной модели

Если модель уже обучена и сохранена, можно загрузить её и сразу делать предсказания.

In [ ]:
# Создаем новый экземпляр и загружаем обученную модель
loaded_model = FinancialForecaster()
loaded_model.load('trained_model.pkl')

# Делаем предсказания
submission2 = loaded_model.predict(
    candles_path=test_candles_path,
    news_path=test_news_path,
    output_path='submission_from_loaded.csv'
)

print("✓ Предсказания сделаны с загруженной моделью")

✓ Модель загружена: trained_model.pkl
ПРЕДСКАЗАНИЕ

[1/4] Загрузка тестовых данных...
  • Загружено 1745 свечей по 19 тикерам
  • Период: 2025-06-02 - 2025-09-08

[2/4] Обработка новостей...
  • Средний sentiment: 0.329

[3/4] Feature engineering...
  • Создано 99 фич

[4/4] Генерация предсказаний...

✓ Submission сохранен: submission_from_loaded.csv
  • Строк: 19
  • Тикеров: 19

✓ ПРЕДСКАЗАНИЕ ЗАВЕРШЕНО
✓ Предсказания сделаны с загруженной моделью
